In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from squeezenet import SqueezeNet, preprocess_input

Using TensorFlow backend.


In [3]:
data_dir = '/home/ubuntu/data/'

In [4]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', 
    target_size=(299, 299),
    batch_size=64
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(299, 299),
    batch_size=128
)

Found 25600 images belonging to 256 classes.
Found 5120 images belonging to 256 classes.


# Model

In [5]:
model = SqueezeNet(weight_decay=1e-5, image_size=299)

# Training

In [6]:
model.compile(
    optimizer=optimizers.SGD(lr=1e-2, momentum=0.9, nesterov=True), 
    loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy']
)

In [7]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=400, epochs=30, verbose=1,
    callbacks=[
        EarlyStopping(monitor='val_acc', patience=4, min_delta=0.01),
        ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2, epsilon=0.007)
    ],
    validation_data=val_generator, validation_steps=40, workers=4
)

Epoch 1/30
400/400 [==============================] - 30s - loss: 2.3677 - acc: 0.4859 - top_k_categorical_accuracy: 0.7056 - val_loss: 1.7769 - val_acc: 0.5992 - val_top_k_categorical_accuracy: 0.8213
Epoch 2/30
400/400 [==============================] - 30s - loss: 1.1757 - acc: 0.7103 - top_k_categorical_accuracy: 0.8941 - val_loss: 1.6904 - val_acc: 0.6111 - val_top_k_categorical_accuracy: 0.8330
Epoch 3/30
400/400 [==============================] - 30s - loss: 0.8528 - acc: 0.7858 - top_k_categorical_accuracy: 0.9343 - val_loss: 1.6566 - val_acc: 0.6285 - val_top_k_categorical_accuracy: 0.8469
Epoch 4/30
400/400 [==============================] - 30s - loss: 0.6804 - acc: 0.8255 - top_k_categorical_accuracy: 0.9533 - val_loss: 1.6594 - val_acc: 0.6283 - val_top_k_categorical_accuracy: 0.8531
Epoch 5/30
400/400 [==============================] - 30s - loss: 0.5532 - acc: 0.8566 - top_k_categorical_accuracy: 0.9666 - val_loss: 1.7166 - val_acc: 0.6404 - val_top_k_categorical_accurac

# Result

In [8]:
val_generator_no_shuffle = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(299, 299),
    batch_size=128, shuffle=False
)

Found 5120 images belonging to 256 classes.


In [9]:
model.evaluate_generator(val_generator_no_shuffle, 40)

[1.6083734691143037, 0.66679687499999996, 0.86289062500000002]